In [91]:
import numpy as np

#Активационная функция сигмоидального нейрона
def sigmoid(x):
    return 1/(1+np.exp(-x))
    
#Производная активационной функции сигмоидального нейрона
def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))

#Вычисляет вектор частных производных целевой функции по каждому из предсказаний(используется при расчете градиента)
def J_quadratic_derivative(y, y_hat):

    assert y_hat.shape == y.shape and y_hat.shape[1] == 1, 'Incorrect shapes'
    
    return (y_hat - y) / len(y)
    
    
#Рассчитывает градиент(аналитическая производная целевой функции)
def compute_grad_analytically(neuron, X, y, J_prime=J_quadratic_derivative):
    
    # Вычисляем активации
    # z - вектор результатов сумматорной функции нейрона на разных примерах
    
    z = neuron.summatory(X)
    y_hat = neuron.activation(z)

    # Вычисляем нужные нам частные производные
    dy_dyhat = J_prime(y, y_hat)
    dyhat_dz = neuron.activation_function_derivative(z)
    
    # осознайте эту строчку:
    dz_dw = X

    

    # а главное, эту:
    grad = ((dy_dyhat * dyhat_dz).T).dot(dz_dw)
    
    # можно было написать в два этапа. Осознайте, почему получается одно и то же
    #2)grad_matrix = dy_dyhat * dyhat_dz * dz_dw
    #2)grad = np.sum(grad_matrix, axis=0)
    
    # Сделаем из горизонтального вектора вертикальный
    
    
    grad = grad.T
    #2)grad.shape = (5,1)
    
    return grad

def compute_error_analytically(neuron, X, y, J_prime=J_quadratic_derivative):
     # Вычисляем активации
    # z - вектор результатов сумматорной функции нейрона на разных примерах
    
    z = neuron.summatory(X)
    y_hat = neuron.activation(z)

    # Вычисляем нужные нам частные производные
    dy_dyhat = J_prime(y, y_hat)
    dyhat_dz = neuron.activation_function_derivative(z)
    

    # а главное, эту:
    error = (dy_dyhat * dyhat_dz).T

    # Сделаем из горизонтального вектора вертикальный
    
    
    error = error.T
    
    return error

class Neuron:

    def __init__(self, weights, activation_function = sigmoid, activation_function_derivative = sigmoid_prime):
        
        assert weights.shape[1] == 1, "Incorrect weight shape"
        
        self.w = weights
        self.activation_function = activation_function
        self.activation_function_derivative = activation_function_derivative
        self.error = 0

    #Сумматорный метод
    def summatory(self, input_matrix):
        return input_matrix.dot(self.w)
        
    #Активационный метод
    def activation(self, summatory_activation):
        #print("s: ", self.activation_function(summatory_activation))
        return self.activation_function(summatory_activation)
        
    #Векторизованный метод рассчета предсказанных значений(гораздо лучший способ)
    def vectorized_forward_pass(self, input_matrix):
        return self.activation(self.summatory(input_matrix))
    
    
    #Обучение весов с помощью одного батча(несколько случайных примеров из всех имеющихся примеров)
    def update_mini_batch(self, X, y, learning_rate, eps):
        
        before = J_quadratic(self, X, y)
        
        #self.w -= learning_rate * compute_grad_analytically(self,X,y)
        self.w -= learning_rate * compute_grad_numerically_2(self,X,y)

        
        return np.abs(before-J_quadratic(self, X, y)) < eps
    
    
    #Обучение весов с помощью батчей
    def SGD(self, X, y, batch_size, learning_rate =0.1, eps=1e-6, max_steps = 200):
        cur_step = 0
        #for _ in range(max_steps):
        while cur_step < max_steps:
            cur_step += 1
            batch = np.random.choice(len(X), batch_size, replace = False)
            if self.update_mini_batch(X[batch], y[batch], learning_rate, eps) == 1: 
                return 1
        return 0
                
    
    def backpropagation(self, X, y, derivative = compute_grad_analytically):
        
        answer = derivative(X,y)
        
        return answer
    
    def backpropagation2(self, X, y, derivative = compute_grad_analytically):
        
        print(np.shape(self.w.reshape(1,len(self.w))))
        answer1 = self.activation_function_derivative(self.summatory(X))
        print("X: ", np.shape(X))
        print("X2: ", np.shape(self.vectorized_forward_pass(X)))
        answer2 = derivative(self, self.vectorized_forward_pass(X), y)
        #answer3 = self.w.reshape(1,len(self.w))
        #answer4 = answer3 * answer2 * answer1
        print(answer2)
        #return answer2


    def teach1(self, X, y, derivative = compute_error_analytically, learning_steps = 10, max_steps = 1e8):
        
        self.error = derivative(self,X,y).reshape(1,len(X))
        answer = np.transpose(self.w.dot(self.error) * self.activation_function_derivative(self.summatory(X)).reshape(1,len(X)))
        #self.w -= answer
        return self.error
    
    def teach2(self, error, weights, X, learning_steps = 10, max_steps = 1e8 ):
        
        #answer1 = self.w.reshape(1,len(self.w))

        self.error = np.reshape(weights.dot(error.reshape(1,len(X))) * self.activation_function_derivative(self.summatory(X)).reshape(1,len(X)), (len(X),1))
        answer = self.w*self.error.reshape(1,len(X))*self.activation_function_derivative(self.summatory(X)).reshape(1,len(X))
        #self.w -= answer
        return self.error
    
def get_error(deltas, sums, weights):

    return (weights.T.dot(deltas.reshape(1,deltas.shape[0])) * sigmoid_prime(sums.T)).mean(axis = 1)#более быстрое решение
    #return (deltas.dot(weights) * sigmoid_prime(sums)).mean(axis = 0).T
    
'''def FindAnswer(l1, l2, l3, y, learning_steps = 1):

    step = 0
    while step < learning_steps:
        step += 1
        #for i in range(len(l2)):
        #    decide2[i] = l2[i].vectorized_forward_pass(l1).reshape(1,len(l1))
        decide2 = np.array([(l2[i].vectorized_forward_pass(l1)) for i in range(len(l2))]).reshape(len(l2),len(l1)).T
        #print(decide2)
        decide3 = l3.vectorized_forward_pass(decide2)
        learning3 = l3.teach1(decide2, y)

        errorl2 = get_error(learning3.reshape(7,1), np.hstack((l2[0].summatory(l1),l2[1].summatory(l1))), l3.w.T)
        print("error: \n",errorl2)
        learning2 = np.array([l2[i].teach2(learning3, l3.w[0], l1) for i in range(len(l2))]).reshape(len(l2),len(l1))
        #return layer3.teach1(np.array([(l2[i].teach2(x,y[i])) for i in range(len(l2))]).T, y[2], y).reshape(len(l1),1)
    return decide3'''

def FindWeight(l1,l2,l3,y):
    weights = np.hstack((l2[0].w,l2[1].w))
    sumsL2 = l2.summatory(l1)

    
w = np.random.randint(-5,5,(2,3))
w = np.array([
                [[0.79], [0.44], [0.85]],
                [[0.85], [0.43], [0.29]]
            ])

w1 = np.array([[0.5],[0.52]])
w = np.array([[[2],[2],[2]], [[2],[2],[2]]])
x = np.array([[0,0,0],
               [0,0,1],
               [0,1,0],
               [1,0,0],
               [0,1,1],
               [1,1,0],
               [1,1,1]])
#x = np.array([[1,1,0]])
x = np.array([[0,1,2]])
y = np.array([[0],[1],[0],[0],[1],[0],[1]])
#y = np.array([[0]])
y = np.array([[1]])
#layer2 = np.array([ Neuron(w[0]),  Neuron(w[1])])
print("w: \n", w.shape)
layer2 = Neuron(w)
layer3 = Neuron(w1)

#print(FindAnswer(x, layer2, layer3, y))
print(FindWeight(x, layer2, layer3, y))
#print(layer2[0].backpropagation2(x, y[0]))

w: 
 (2, 3, 1)


AssertionError: Incorrect weight shape